In [544]:
# pip install giotto-tda
# pip install gudhi

In [545]:
import pandas as pd
import numpy as np
import matplotlib as plt
from gtda.time_series import SingleTakensEmbedding, TakensEmbedding
import gudhi
from gtda.plotting import plot_point_cloud

series = np.log(pd.read_csv("SP500.csv", header=None).to_numpy().squeeze())
time_series = np.array([])

In [546]:
# initial values for important variables -- will have to find optimal values later on

n = 4 # dim of vectors
d = 5 # time delay
w = 5 # window size
epsilon = 0.1 # resolution threshold
q = 3 # number of precision qubits
simplex_dim = 2 # maximum simplex tree dimension used

In [547]:
def TakenEmbedding(time_series, dim, delay):
    serie = time_series.reshape(1, -1)
    STE = TakensEmbedding(time_delay=delay, dimension=dim)
    return STE.fit_transform(serie)[0]

In [548]:
def TakenPointCloud(embedding, window_size):
    point_cloud = []
    l = len(embedding)
    for i in range(l - window_size):
        window = embedding[i : i + window_size]
        point_cloud.append(window)

    return np.array(point_cloud)

Following Takens' embedding theorem, transform the `time_series` into a series of $ N $-dimensional vectors. Afterward, apply a sliding window to these vectors and obtain a time-varying point cloud.

In [549]:
embedding = TakenEmbedding(series, n, d)
type(embedding[0])

numpy.ndarray

In [550]:
point_clouds = TakenPointCloud(embedding, w)
# a numpy vector of numpy vectors of numpy vectors containing all the different point clouds.
print(point_clouds[0])
# point_cloud[t] gives the point cloud at time t

[[7.13389833 7.17081353 7.16110014 7.15205153]
 [7.14663437 7.1664011  7.14610857 7.14628387]
 [7.15787955 7.15850232 7.14739207 7.15252724]
 [7.15000317 7.15946102 7.14883305 7.16494271]
 [7.15460754 7.16732333 7.15964177 7.16181205]]


In [551]:
def SimplexTrees(clouds, simplices, e):
    tree = []

    for i in range(len(clouds)):
        rips_complex = gudhi.RipsComplex(points = clouds[i], max_edge_length = e)
        simplex_tree = rips_complex.create_simplex_tree(max_dimension = simplices)

        simplex = [val for val, dist in simplex_tree.get_filtration()]

        simplex = sorted(simplex, key=lambda vector: (len(vector), vector[0], vector[len(vector) - 1]))

        tree.append(simplex)

    return tree

In [552]:
simplex_trees = SimplexTrees(point_clouds, simplex_dim, epsilon)
print(simplex_trees[0])
# simplex_trees[t] gives the simplex trees for the point cloud at time t

[[0], [1], [2], [3], [4], [0, 1], [0, 2], [0, 3], [0, 4], [1, 2], [1, 3], [1, 4], [2, 3], [2, 4], [3, 4], [0, 1, 2], [0, 1, 3], [0, 2, 3], [0, 1, 4], [0, 3, 4], [0, 2, 4], [1, 2, 3], [1, 2, 4], [1, 3, 4], [2, 3, 4]]


In [559]:
def BoundaryOperators(simplex_tree, dimension):
    points = len([elem for elem in simplex_tree if len(elem) == 1])
    forms = [elem for elem in simplex_tree if len(elem) == dimension + 1]

    operator = []

    for j in range(len(forms)):
        col = [1 if num in forms[j] else 0 for num in range(points)]

        count = 0

        for j in range(len(col)):
            if(col[j] == 1):
                col[j] = pow(-1, count)
                count += 1

        operator.append(col)

    return [[operator[j][i] for j in range(len(operator))] for i in range(len(operator[0]))] 
# if we want each operator[i] to correspond to the ith point

    return operator
# if we want each operator[i] to correspond to the ith simplex

In [562]:
print(BoundaryOperators(simplex_trees[0], 2))

[[1, 1, 1, 1, 1, 1, 0, 0, 0, 0], [-1, -1, 0, -1, 0, 0, 1, 1, 1, 0], [1, 0, -1, 0, 0, -1, -1, -1, 0, 1], [0, 1, 1, 0, -1, 0, 1, 0, -1, -1], [0, 0, 0, 1, 1, 1, 0, 1, 1, 1]]
